# <p style="color:#1BD1E7; font-size:1.5em">**Projeto Aplicado em Ciência de Dados II**</p> 
## <p style="color:#1BD1E7; font-size:1em">**Clusters de performance na NBA**</p> 

---

* António Teotónio - 111283

* João Batista - 111611

* João Dias - 110305

* Felipe Pereira - 110861

* David Franco - 110733

--- 

### Import das bibliotecas utilizadas

In [3]:
from selenium import webdriver
import time
from selenium.webdriver.chrome.service import Service
import pandas as pd
from selenium.webdriver.common.by import By

path = './chromedriver.exe'

### Execução

Este notebook faz o web scraping de todas as estatísticas dos jogadores na página do Basketball Reference (Daily Leaders) por mês. O output é um ficheiro csv com a performance de todos os jogadores em todos os jogos do mês em questão. Deve ser executado tantas vezes quantos meses se queiram extrair (com os devidos ajustes nas variáveis). Todos os datasets gerados foram guardados na pasta datasets_23_24_season
- Para executar é necessário descarregar o chromedriver tendo em conta a versão do Google Chrome que está a ser utilizada. Pode ver-se mais informação aqui: https://developer.chrome.com/docs/chromedriver/downloads
- De seguida é necessário ajustar manualmente as variáveis month, day_i, year. 
- Ajustar o número de dias dentro do for loop a scrapear `for day in range(day, day+32)`

In [ ]:
# Cria uma instância do Service com o caminho do chromedriver
service = Service(executable_path=path)
# Inicializa o driver passando o serviço
driver = webdriver.Chrome(service=service)

# É necessário ajustar estas variáveis manualmente
month = 3
year = 2024
day_i = 1
day = day_i

# URLs a fazer scraping
urls = []
for day in range(day, day+32):
    urls.append(f"https://www.basketball-reference.com/friv/dailyleaders.fcgi?month={month}&day={day}&year={year}&type=all")

# Criar uma lista para armazenar os dados
data = []

# Aceder a cada URL e realizar o scraping
for i, url in enumerate(urls):
    driver.get(url)
    time.sleep(5)  # Esperar 5 segundos para a página carregar
    
    # Obter o dia atual
    day_f = day_i
    current_day = day_f + i  

    try:
        # Localizar a tabela na página
        table = driver.find_element(By.XPATH, '//table[@id="stats"]')  
        
        # Extrair os dados da tabela
        rows = table.find_elements(By.TAG_NAME, 'tr')
        
        for row in rows:
            cols = row.find_elements(By.TAG_NAME, 'td')
            # Verificar se há colunas na linha
            if len(cols) > 0:
                row_data = [col.text for col in cols]
                date = f"{year}-{month:02d}-{current_day:02d}"  
                row_data.append(date)
                data.append(row_data) 
                print(row_data)  

    except Exception as e:
        print(f"Erro ao acessar a tabela na URL {url}: {e}")

# Criar um DataFrame com os dados recolhidos
columns = ["Player", "Tm", "vs", "Opp", "W/L", "MP", "FG", "FGA", "FG%", "3P", "3PA", "3P%", "FT", "FTA", "FT%", "ORB", "DRB", "TRB", "AST", "STL", "BLK", "TOV", "PF", "PTS", "+/-", "GmSc", 'Date']
df = pd.DataFrame(data, columns=columns)

print(df)

# Fechar o driver
driver.quit()


In [6]:
# Guarda o df num csv
df.to_csv('dailyleaders.csv', index=False)